In [35]:
import warnings

In [36]:
warnings.filterwarnings('ignore')

In [37]:
import pandas as pd 

In [38]:
data=pd.read_csv(r'C:\Users\deyar\OneDrive\Desktop\InsurencepPricePrediction\notebooks\data\insurance.csv')

In [39]:
data.head(2)

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55


In [40]:
cat=[]
num=[]
for i in data.columns:
    if data[i].dtype=='object':
        cat.append(i)
    else:
        num.append(i)

In [41]:
data_cat=data[cat]

In [42]:
data_num=data[num]

In [43]:
from sklearn.impute import SimpleImputer

In [44]:
from sklearn.pipeline import Pipeline

In [45]:
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder

In [46]:
from sklearn.compose  import ColumnTransformer

In [47]:
num

['age', 'bmi', 'children', 'expenses']

In [48]:
num.remove('expenses')

In [49]:
num

['age', 'bmi', 'children']

In [50]:
cat

['sex', 'smoker', 'region']

In [51]:
y=data.iloc[:,-1:]

In [52]:
x=data.drop('expenses',axis=1)

In [53]:
for  j in data_cat.columns:
    data_cat[j]=data_cat[j].astype('category').cat.codes

In [54]:
data=pd.concat([data_cat,data_num],axis=1)

In [55]:
data.head(2)

,sex,smoker,region,age,bmi,children,expenses
0,0,1,3,19,27.9,0,16884.92
1,1,0,2,18,33.8,1,1725.55


In [56]:
data_cat=data[cat]

In [57]:
data_num=data[num]

In [58]:
data_num.head(2)

,age,bmi,children
0,19,27.9,0
1,18,33.8,1


In [59]:
from scipy.stats import kendalltau

In [60]:
s=[]
for i in data_cat.columns:
   coef,p=kendalltau(data_cat[i],y)
   if p<0.05:
    s.append('dependent')
   else:
     s.append("independent")

In [61]:
m=data_cat.columns
x1=pd.DataFrame(s,columns=['result'])
x2=pd.DataFrame(m,columns=['columns'])
pd.concat([x1,x2],axis=1)

,result,columns
0,independent,sex
1,dependent,smoker
2,independent,region


for sex and region expenses will not be effected

In [62]:
from scipy.stats import spearmanr

In [63]:
o=[]
for j in data_num.columns:
    coef,p=spearmanr(data_num[j],y)
    if p<0.5:
        o.append('dependent')
    else:
        o.append('independent')

In [64]:
x1=pd.DataFrame(o,columns=['result'])
n=data_num.columns
x2=pd.DataFrame(n,columns=['columns'])
pd.concat([x1,x2],axis=1)

,result,columns
0,dependent,age
1,dependent,bmi
2,dependent,children


In [65]:
data.corr()['expenses']

sex         0.057292
smoker      0.787251
region     -0.006208
age         0.299008
bmi         0.198576
children    0.067998
expenses    1.000000
Name: expenses, dtype: float64

see sex and region both are not correlated with expenses

In [66]:
numerical_pipeline = Pipeline(steps=[
                    ("impute", SimpleImputer(strategy='median')),
                    ("scale", MinMaxScaler())
                    ])

category_pipeline = Pipeline(steps=[
                    ("impute", SimpleImputer(strategy="most_frequent")),
                    ("encoder", OneHotEncoder(sparse_output=False, drop='first')),
                    ("scale", MinMaxScaler())
                    ])
preprocessor=ColumnTransformer([
            ('num_pipeline',numerical_pipeline,num),
            ('cat_pipeline',category_pipeline,cat)
            ])
            


In [67]:
preprocessor.fit_transform(x)

array([[0.02173913, 0.32075472, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.47978437, 0.2       , ..., 0.        , 1.        ,
        0.        ],
       [0.2173913 , 0.45822102, 0.6       , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.        , 0.56334232, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.06521739, 0.26415094, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.93478261, 0.35309973, 0.        , ..., 1.        , 0.        ,
        0.        ]])